# 1. Create Output File

Firstly, input and output files are setup and opened.

- testTrack_hierarchy.txt is the testing file and is in the format UserID|trackID|Album|Artist|Genre1|Genre2|Genre3 | ..... It is 120,000 entries long.
- trainIdx2_matrix.txt is the training file and is in the format of userID|itemID|score. It is over 12,400,000 entries long.
- Then, we create output1.txt file, which contains the user_id, track_id, and album rating and artist rating for a given user and track. It has 120,000 entries and there will be 6 entries for each user.


In [1]:
import numpy

dataDir = '/Users/hinalpatel/Desktop/Big data analyst/HW5_YahooMusic/data_in_matrixform/'
file_name_test = dataDir + 'testTrack_hierarchy.txt'
file_name_train = dataDir + 'trainIdx2_matrix.txt'
output_file = dataDir + 'output1.txt'

# Open files
fTest = open(file_name_test)
fTrain = open(file_name_train)
TrainLine = fTrain.readline()
fOut = open(output_file, 'w')

In [2]:
# Initialize arrays to hold trackID, albumID, and artistID for all 6 entries of a user.
trackID_vec = [0] * 6
albumID_vec = [0] * 6
artistID_vec = [0] * 6
lastUserID = -1

# Initialize numpy array to hold album and artist ratings for each user
user_rating_in_train = numpy.zeros(shape=(6,3))

- First, We go through each line of the testing file, we take each element from a line and create a list. 
- We then grab the user_id, track_id, album_id, and artist_id from that list.
- Next, we check if a new user has been reached (we enter this if statement from the start since last_user_id starts as -1).

In [3]:
for line in fTest:
    arr_test = line.strip().split('|')
    userID = arr_test[0]    
    trackID = arr_test[1]   
    albumID = arr_test[2]  
    artistID = arr_test[3]  
    
    if userID != lastUserID:    # if new user reached
        ii = 0                     # reset ii
        user_rating_in_train = numpy.zeros(shape=(6,3))    # reset this array
        
    trackID_vec[ii] = trackID
    albumID_vec[ii] = albumID
    artistID_vec[ii] = artistID
    ii = ii + 1
    lastUserID = userID
     # if last entry for current user reached
    if ii == 6:                    
        while(TrainLine):
            #for TrainLine in fTrain:
                arr_train = TrainLine.strip().split('|')   
                trainUserID = arr_train[0]
                trainItemID = arr_train[1]
                trainRating = arr_train[2]
                TrainLine = fTrain.readline()   

                if trainUserID < userID:
                    continue   
                if trainUserID == userID:
                    # go through each row in (6,3) user_rating_in_train array for current row in training file
                    for nn in range(0, 6):   
                         # if album found
                        if trainItemID == albumID_vec[nn]:    
                            user_rating_in_train[nn, 0] = trainRating    # set album rating
                         # if artist found
                        if trainItemID == artistID_vec[nn]:   
                            user_rating_in_train[nn, 1] = trainRating    # set artist rating
                if trainUserID > userID:   
                    for nn in range(0, 6):    
                        out_str = str(userID) + '|' + str(trackID_vec[nn]) + '|' + str(user_rating_in_train[nn,0]) + '|' + str(user_rating_in_train[nn, 1])
                        fOut.write(out_str + '\n')
                    break    
fTest.close()
fTrain.close()
fOut.close()


In [4]:
# Close all the files
fTest.close()
fTrain.close()
fOut.close()

- Using the output file from sample code, we construct a predictions file that will include the predictions for each user in the format userID_trackID,prediction. 
- Predictions will be either 1 or 0, with 1 indicating that we predict the user will like the music and 0 indicating that we predict the user will not like the music (or vice versa).
- There will be 3-1 predictions and 3-0 predictions for each user, with a total of 6 predictions (6 songs) for each user.
- For each user's six tracks, we analyze album and artist ratings (if available) and add weights to them.
- If we apply a larger weight to the album score since it is more probable that a user would like a song from the same album than it is to like a song from the same artist.
- After that, We pick the top three ratings and assign these songs a 1 (indicating as the user would like more), while the other three songs are assigned a 0. (user will like these songs less).


In [5]:
# Initialize some values
ratingArray = numpy.zeros(shape=(2,6))
lastUserID = -1
trackID_out_vec = [0] * 6

# Open files
output_file_2 = dataDir + 'prediction.csv'
fOut_2 = open(output_file_2, 'w')
fOut = open(output_file)

# Write header
fOut_2.write('TrackID,Predictor\n')


18

In [6]:
# Go through each line of the output file
for line in fOut:
    arr_out = line.strip().split('|')
    userID_out = arr_out[0]                
    trackID_out = arr_out[1]               
    album_rating = float(arr_out[2]) * .9  
    artist_rating = float(arr_out[3]) * .1  
    
    # if new user reached
    if userID_out != lastUserID:  
        # reset ii
        ii = 0 
         # reset this array
        ratingArray = numpy.zeros(shape=(2,6))   
     # if still on same user (so from 0 to 5 inclusive)    
    if ii < 6:                               
        ratingArray[0, ii] = album_rating    # add album rating to this vector
        ratingArray[1, ii] = artist_rating   # add artist rating to this vector
        trackID_out_vec[ii] = trackID_out    # add trackID to this vector
        
    ii = ii + 1
    lastUserID = userID_out
    # if last entry for current user reached    
    if ii == 6:                                                   
        ratingArray_sum = numpy.zeros(shape=(6))            # initialize numpy array for ratings sums
        numpy.sum(ratingArray, axis=0, out=ratingArray_sum) # sum the album rating and artist rating for each track
        indices = (-ratingArray_sum).argsort()[:3]          # get indices of top 3 scores
        
        # set the predictions by setting a 1 for elements with indices in the indices list 
        # and setting a 0 for elements with indices not in the indices list
        predictions = numpy.zeros(shape=(6))   
        for index in indices:
            predictions[index] = 1    
            
        # write to the output file for the 6 track predictions for the current user
        for nn in range(0, 6):
            out_str = str(userID_out) + '_' + str(trackID_out_vec[nn]) + ',' + str(int(predictions[nn]))
            fOut_2.write(out_str + '\n')



fOut_2.close()
fOut.close()